<a href="https://colab.research.google.com/github/sergioGarcia91/ML_and_EDA/blob/main/07a_YOLOv8_Amonites_20240323.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detección de Objetos - Dataset: Amonites


> *Ser tan rápidos como el más lento, \
> y ser tan lentos como el más rápido.*

**Autor:** Sergio Andrés García Arias  
**Versión 01:** Marzo 2024




# Introducción

En este Notebook, abordaremos el uso de un modelo de machine learning pre-entrenado para la detección de `Amonites`, un objeto de interés específico. Utilizaremos el modelo `YOLOv8`, desarrollado por Ultralytics. La [documentación oficial](https://docs.ultralytics.com/es) de YOLOv8 proporciona información detallada y tutoriales para la modificación de sus modelos.

> *El objetivo principal de este proyecto es reentrenar una Red Neuronal Convolucional (CNN) para que pueda detectar exclusivamente los nuevos objetos de interés, en este caso, los amonites.*


---
**Observación:** En la propia documentación y en la web se encuentran videos que ejemplifican este proceso. Todo el proceso preliminar de generación de las imágenes para el entrenamiento no se va a ejemplificar. Esta primera etapa de obtención de imágenes y generación del dataset fue realizada utilizando la herramienta suministrada por Ultralytics: [Roboflow](https://roboflow.com). Dos videos que sintetizan el trabajo de este Notebook son:

- [How to train YOLOv8 Object Detection on Custom Dataset | step by step Tutorial | Google Colab](https://www.youtube.com/watch?v=ZzC3SJJifMg&t=677s)
- [YOLOV8: Object Detection Annotation | Annotate Custom Data using Roboflow](https://www.youtube.com/watch?v=qn96xC3LV2Y)


# Uso de GPU

Para lograr un rendimiento óptimo y acelerar el entrenamiento de modelos de Redes Neuronales Convolucionales (CNN), es recomendable utilizar una unidad de procesamiento gráfico (GPU). La capacidad de procesamiento paralelo de las GPU permite realizar cálculos intensivos de manera más eficiente que las unidades de procesamiento central (CPU) tradicionales.

Antes de comenzar el entrenamiento, es importante cambiar el entorno de ejecución para que utilice una GPU. Esto se puede hacer seleccionando:

- `Entorno de ejecución` y luego `Cambiar tipo de entorno de ejecución`.



<center><img src='https://github.com/sergioGarcia91/ML_and_EDA/blob/main/yolo01.png?raw=true' width=500 /></center>

# Limitaciones de recursos

Es posible que te encuentres con la situación en la que no puedas acceder al uso de la `GPU`. Esto se puede manifestar a través de un mensaje similar al de la imagen.

> *Puedes obtener información sobre las limitaciones de este entorno virtual en las [Preguntas frecuentes de Colaboratory](https://research.google.com/colaboratory/faq.html#usage-limits).*

> *Hasta la fecha de la réplica de este Notebook (2024-03-25), no he podido acceder a una GPU. En fechas anteriores sí fue posible; simplemente es cuestión de tener paciencia hasta obtener una. Google ofrece la opción de pago si se va a hacer uso continuo de una GPU o recursos superiores, lo cual puede ser de interés; sin embargo, para fines ilustrativos, tener algo de paciencia y suerte es suficiente.*


<center><img src='https://github.com/sergioGarcia91/ML_and_EDA/blob/main/yolo02.png?raw=true' width=500 /></center>



# Inicio


En la web de Ultralytics, se proporciona una ejemplificación de cómo realizar la [Detección de objetos](https://docs.ultralytics.com/tasks/detect/#val). Algunos comentarios en inglés se mantienen tal como están en la documentación original, nuestra tarea es verificar que se ejecute correctamente y realizar los ajustes necesarios si es requerido. También incluiremos cualquier otra acción que sea de nuestro interés durante el proceso.


In [ ]:
# Instalar ultralytics
!pip install ultralytics

In [ ]:
!pip install tqdm --upgrade

In [ ]:
# Importar librerias requeridas
import os
import shutil
import random

from tqdm.notebook import tqdm

# Drive

In [ ]:
# Conectar al Drive para cargar los Amonites
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Consideraciones previas

En Roboflow se delimitaron los objetos de interés, en nuestro caso, los `Amonites`, de modo que el modelo detectará solo una clase de objeto. El dataset consta de 121 imágenes, de las cuales se utilizarán 100 para el entrenamiento. De estas 100 imágenes, el $80\%$ se utilizó para el entrenamiento y el $20\%$ restante para la validación. La evaluación del modelo se realizó utilizando las 21 imágenes restantes que el modelo nunca ha visto.


<center><img src='https://github.com/sergioGarcia91/ML_and_EDA/blob/main/yolo03.png?raw=true' width=1000 /></center>

### Ajuste del archivo YAML

Antes de poder utilizar nuestro dataset preparado en Roboflow, es necesario ajustar el archivo `data.yaml` descargado del archivo comprimido. Este archivo con extensión `.yaml` se puede abrir con un editor de texto, y se deben modificar las rutas de `train:` y `val:`. La ruta de `test:` no es necesaria en este punto ya que la configuraremos manualmente más adelante. En mi caso, las rutas de entrenamiento y validación corresponden a las rutas en mi Google Drive.

<center><img src='https://github.com/sergioGarcia91/ML_and_EDA/blob/main/yolo04.png?raw=true' width=700 /></center>



# Cargar modelo base

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.32 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/78.2 GB disk)


In [ ]:
# Load a model
model = ultralytics.YOLO('yolov8n.yaml')  # build a new model from YAML
model = ultralytics.YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = ultralytics.YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

Transferred 355/355 items from pretrained weights


## Entrenar

A partir de este punto, después de cargar el modelo y entrenarlo, es posible que en el panel izquierdo, en la sección de `Archivos` identificada con el símbolo de la carpeta, se cree una nueva carpeta donde se almacenarán algunos archivos importantes para observar el comportamiento del modelo.


In [ ]:
# en pathYaml se coloca la ruta del archivo yaml generado al descargar las
# imágenes ya listas para el entrenamiento de Robotflow
pathYaml = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Ammonite.v2i.yolov8/data.yaml'

# Train the model
results = model.train(data=pathYaml, epochs=200, imgsz=640)

## Predecir

In [ ]:
# Predecir
# Load a model
pathModelo = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Resultados/runs/detect/train6/weights/best.pt'
model = ultralytics.YOLO(pathModelo)  # load a custom model

# Predict with the model
pathTest = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/21_imag_test'
results = model.predict(pathTest, conf=0.05, imgsz=640)  # predict on an image

### Guardar figuras

In [ ]:
# Process results list
# Para guardar los resultados de las 21 imágenes que no ha visto la CNN
id = 1
pathSave = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Resultados/'
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen
    result.save(filename=pathSave+f'result_{id}.jpg')  # save to disk
    id += 1

Algunos resultados:

<img src="https://github.com/sergioGarcia91/ML_and_EDA/blob/main/result_1.jpg?raw=true" alt="Amonite 1" width="500" >
<img src="https://github.com/sergioGarcia91/ML_and_EDA/blob/main/result_2.jpg?raw=true" alt="Amonite 2" width="500" >
<img src="https://github.com/sergioGarcia91/ML_and_EDA/blob/main/result_12.jpg?raw=true" alt="Amonite 12" width="500" >
<img src="https://github.com/sergioGarcia91/ML_and_EDA/blob/main/result_21.jpg?raw=true" alt="Amonite 21" width="500" >




### Videos

Ahora, lo interesante es ver el resultado de dos videos que utilicé:

1. El primer video es una grabación de la pantalla de mi [Navegador](https://www.youtube.com/watch?v=77b0GASeQn8) en la cual realicé una consulta en Google sobre `Amonites`.
2. El segundo video está grabado con mi [Celular](https://www.youtube.com/shorts/M7yWYOKWsNM) que muestra algunas muestras que tengo, incluyendo unos `Amonites`.


In [ ]:
# Predecir videos
# Load a model
pathModelo = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Resultados/runs/detect/train6/weights/best.pt'
model = ultralytics.YOLO(pathModelo)  # load a custom model

# Predict with the model
pathVideo1 = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/navegador.mkv'
results = model.predict(pathVideo1, conf=0.35, imgsz=640, save=True)  # predict on an image

In [ ]:
# Predecir videos
# Load a model
pathModelo = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Resultados/runs/detect/train6/weights/best.pt'
model = ultralytics.YOLO(pathModelo)  # load a custom model

# Predict with the model
pathVideo2 = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/celular.mp4'
results = model.predict(pathVideo2, conf=0.05, imgsz=640, save=True)  # predict on an image

El resultado puede considerarse aceptable ya que logra detectar los amonites. Sin embargo, no logra detectar todos, lo cual puede atribuirse principalmente a que se realizó un entrenamiento con solo 100 imágenes. Para mejorar la detección, sería necesario aumentar la cantidad de imágenes utilizadas en el entrenamiento, y para ello se puede emplear la técnica de aumento de datos. Esta opción de aumento de datos está disponible directamente en Roboflow.

---
Algunas recomendaciones adicionales para mejorar la detección de objetos:

1. **Variedad en los datos:** Asegúrate de incluir una amplia variedad de imágenes que representen diferentes aspectos y variaciones de los objetos, como diferentes ángulos, tamaños, colores de fondo, iluminación, etc. Esto ayudará al modelo a generalizar mejor y detectar los amonites en diversas condiciones.

2. **Calidad de las imágenes:** Utiliza imágenes de alta calidad y resolución para el entrenamiento. Imágenes borrosas o de baja calidad pueden afectar la capacidad del modelo para detectar correctamente los objetos. Aunque tal vez usar imágenes de baja calidad sea posible, estas deben ser lo suficientemente representativas para identificar el objeto.

3. **Etiquetado preciso:** Asegúrate de etiquetar correctamente las imágenes de entrenamiento para que el modelo aprenda con precisión las características de los objetos.

4. **Ajuste de hiperparámetros:** Experimenta con diferentes hiperparámetros durante el entrenamiento, como el tamaño del lote, la tasa de aprendizaje, el número de épocas, etc. Estos ajustes pueden tener un impacto significativo en el rendimiento del modelo.

5. **Validación cruzada:** Utiliza técnicas de validación cruzada para evaluar el rendimiento del modelo de manera más rigurosa y garantizar que no esté sobreajustado a los datos de entrenamiento.

6. **Revisión del modelo:** Después del entrenamiento, realiza una revisión exhaustiva del modelo para identificar cualquier área de mejora o posibles errores en la detección. Ajusta y vuelve a entrenar el modelo según sea necesario.





# Exportar a TFlite

In [ ]:
!pip install 'tensorflow<2.14.0' ultralytics

In [ ]:
import ultralytics

# Load a model
pathModelo = '/content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Resultados/runs/detect/train6/weights/best.pt'
model = ultralytics.YOLO(pathModelo)  # load a custom model

# Export the model
model.export(format='tflite')

# Copiar carpetas

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!cp -r /content/runs /content/drive/MyDrive/UIS/ASIGNATURAS/Notebooks_Python/Yolo_Amonite/Resultados

# Extras

In [ ]:
# Predecir imagen web
# Load a model
model = ultralytics.YOLO('/content/runs/detect/train6/weights/best.pt')  # load a custom model

# Predict with the model
urlImag = 'https://whalerslocker.com/cdn/shop/articles/6.5_-Ammonite-Fossil-Split-Pair_1200x1200.jpg'
results = model.predict(urlImag, conf=0.35, imgsz=640, save=True, show=True)  # predict on an image

# Fin